In [43]:
%pip install --upgrade openai plotly scikit-learn pymilvus numpy
%load_ext dotenv
%dotenv

Defaulting to user installation because normal site-packages is not writeable
^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install --upgrade openai plotly scikit-learn pymilvus numpy
%load_ext dotenv
%dotenv

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 5.7 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
%pip install --upgrade openai plotly scikit-learn pymilvus numpy
%load_ext dotenv
%dotenv

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 5.7 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [1]:
import json
import pandas as pd
import re
import os
from os import walk
from openai.embeddings_utils import get_embedding, get_embeddings
import openai
openai.api_key = os.environ['OPENAI_API_KEY']


fhir_txt_filenames = []
for (dirpath, dirnames, filenames) in walk("narrative"):
    fhir_txt_filenames.extend([f for f in filenames])
fhir_txt_filenames=sorted(fhir_txt_filenames)

CHUNK_MAX = 3000
def chunk_file(filename, contents):
    contents_clamped = []
    for c in contents:
        for i in range(0, len(c), CHUNK_MAX):
            contents_clamped.append(c[i*CHUNK_MAX:(i+1)*CHUNK_MAX])
    contents = contents_clamped
    def new_chunk(c):
        chunks.append(c)
    chunks = []
    new_chunk("")
    start_line = 0

    for c in contents[start_line:]:
        if len(chunks[-1]) + len(c) > CHUNK_MAX:
            new_chunk(c)
        else:
            chunks[-1] += c
    return chunks

filename_to_chunks = {}
for fname in fhir_txt_filenames:
    with open(f'./narrative/{fname}') as fhandle:
        contents = fhandle.readlines()
        chunks = chunk_file(fname, contents)
        filename_to_chunks[fname] = chunks

with open("./chunks.json", "w") as chunks_file:
    json.dump(filename_to_chunks, chunks_file, indent=2)

# TODO: batch requests across filenames
with open("./simpler-embedded.json", "w") as embedded_json:
    for filename in fhir_txt_filenames:
        embeddings = []
        chunks = filename_to_chunks[filename]
        if len(chunks) == 0: continue
        # print(chunks)
        print("Call", filename)
        try: 
            embed_data = openai.Embedding.create(input = chunks, model="text-embedding-ada-002")['data']
            chunks_embedded = [e['embedding'] for e in embed_data]
            for v in [[filename, i, embedding] for i, embedding in enumerate(chunks_embedded)]:
                json.dump(v, embedded_json)
                embedded_json.write("\n")
        except:
            print("Err did not", filename)
            pass
        print("Embedded", filename)


Call 1.info.txt
Embedded 1.info.txt
Call 10.info.txt
Embedded 10.info.txt
Call 100.info.txt
Embedded 100.info.txt
Call 1000.info.txt
Embedded 1000.info.txt
Call 1001.info.txt
Embedded 1001.info.txt
Call 1002.info.txt
Embedded 1002.info.txt
Call 1003.info.txt
Embedded 1003.info.txt
Call 1004.info.txt
Embedded 1004.info.txt
Call 1005.info.txt
Embedded 1005.info.txt
Call 1006.info.txt
Embedded 1006.info.txt
Call 1007.info.txt
Embedded 1007.info.txt
Call 1008.info.txt
Embedded 1008.info.txt
Call 1009.info.txt
Embedded 1009.info.txt
Call 101.info.txt
Embedded 101.info.txt
Call 1010.info.txt
Embedded 1010.info.txt
Call 1011.info.txt
Embedded 1011.info.txt
Call 1012.info.txt
Embedded 1012.info.txt
Call 1013.info.txt
Embedded 1013.info.txt
Call 1014.info.txt
Embedded 1014.info.txt
Call 1015.info.txt
Embedded 1015.info.txt
Call 1016.info.txt
Embedded 1016.info.txt
Call 1017.info.txt
Embedded 1017.info.txt
Call 1018.info.txt
Embedded 1018.info.txt
Call 1019.info.txt
Embedded 1019.info.txt
Call 1

In [ ]:
fhir_milvus.release()
fhir_milvus.drop_index()

index = {
    "index_type": "FLAT",
    "metric_type": "L2",
    "params": {},
}
fhir_milvus.create_index(field_name="embeddings", index_params=index)
# fhir_milvus.load()


Status(code=0, message=)

In [ ]:
# 

In [ ]:
fhir_milvus.load()
search_params = {"metric_type": "L2", "params": {}, "offset": 0}

query_embedding = openai.Embedding.create(input = ["fhir patient identifier"], model="text-embedding-ada-002")['data'][0]['embedding']

results = fhir_milvus.search(
	data=[query_embedding], 
	anns_field="embeddings", 
	param=search_params,
	output_fields=['filename', 'chunk'],
	limit=5, 
	expr="core==true",
)[0]

for hit in results:
	print(hit.entity)
	chunk = filename_to_chunks[hit.entity.get('filename')][hit.entity.get('chunk')]
	print(chunk)

id: 14421, distance: 0.32674795389175415, entity: {'filename': 'fhirpath.txt', 'chunk': 12}
fhirpath.txt
generated on Sat, May 28, 2022 12:58+1000.
Links: Search [] Version History Table of Contents QA Page |
Compare to R4 [] [CC0] Propose a change []

id: 3767, distance: 0.3538467586040497, entity: {'filename': 'qa.txt', 'chunk': 47}
qa.txt
 http://hl7.org/fhir/ig/vhdir/Organization/independencerehab in
 orgrole-example-services (Possible Ids: hl7, 1,
 1832473e-2fe0-452d-abe9-3cdb9879522f, 2, 2.16.840.1.113883.19.5,
 f001, f002, f201, f203, f003, mmanu, 3, hl7pay, )
 orgrole-example-services) Unable to resolve invalid example
 reference http://hl7.org/fhir/ig/vhdir/Network/patriotppo in
 orgrole-example-services

Resource Patient

 INFORMATION:Patient.generalPractitioner (example
 patient-example-infant-mom) Unable to resolve example reference to
 Practitioner/21B in patient-example-infant-mom (Possible Ids:
 example, xcda-author, 3ad0687e-f477-468c-afd5-fcc2bf897809, f001,
 f002, f00

In [ ]:
import json
import pandas as pd
import re
import os
from os import walk
from openai.embeddings_utils import get_embedding, get_embeddings
import openai
openai.api_key = os.environ['OPENAI_API_KEY']


fhir_txt_filenames = []
for (dirpath, dirnames, filenames) in walk("./txt-simple/"):
    fhir_txt_filenames.extend([f for f in filenames])
    break

CHUNK_MAX = 4000
def chunk_file(filename, contents):
    contents_clamped = []
    for c in contents:
        for i in range(0, len(c), CHUNK_MAX):
            contents_clamped.append(c[i*CHUNK_MAX:(i+1)*CHUNK_MAX])
    contents = contents_clamped
    def new_chunk(c):
        chunks.append(f'{filename}\n{c}')
    chunks = []
    new_chunk("")
    start_line = 0
    for i, l in enumerate(contents):
        if re.match("^(\d+\.\d+)|Raw JSON", l):
            start_line = i
            break

    for c in contents[start_line:]:
        if len(chunks[-1]) + len(c) > CHUNK_MAX or re.match("^(\d+\.\d+)|Raw JSON", c):
            new_chunk(c)
        else:
            chunks[-1] += c
    return chunks[1:]

filename_to_chunks = {}
for fname in fhir_txt_filenames:
    with open(f'./txt-simple/{fname}') as fhandle:
        contents = fhandle.readlines()
        chunks = chunk_file(fname, contents)
        filename_to_chunks[fname] = chunks

with open("./chunks.json", "w") as chunks_file:
    json.dump(filename_to_chunks, chunks_file, indent=2)

with open("./simpler-embedded.json", "w") as embedded_json:
    for filename in fhir_txt_filenames:
        embeddings = []
        chunks = filename_to_chunks[filename]
        if len(chunks) == 0: continue
        # print(chunks)
        print("Call", filename)
        embed_data = openai.Embedding.create(input = chunks, model="text-embedding-ada-002")['data']
        chunks_embedded = [e['embedding'] for e in embed_data]
        for v in [[filename, i, embedding] for i, embedding in enumerate(chunks_embedded)]:
            json.dump(v, embedded_json)
            embedded_json.write("\n")
        print("Embedded", filename)


Call valueset-all-distance-units.txt
Embedded valueset-all-distance-units.txt
Call sc-valueset-medicationknowledge-status.json.txt
Embedded sc-valueset-medicationknowledge-status.json.txt
Call valueset-encounter-type.json.txt
Embedded valueset-encounter-type.json.txt
Call valueset-condition-stage.txt
Embedded valueset-condition-stage.txt
Call valueset-verificationresult-status.json.txt
Embedded valueset-verificationresult-status.json.txt
Call catalogentry-definitions.txt
Embedded catalogentry-definitions.txt
Call healthcareservice-mappings.txt
Embedded healthcareservice-mappings.txt
Call hlaresult.txt
Embedded hlaresult.txt
Call medicationadministration-mappings.txt
Embedded medicationadministration-mappings.txt
Call valueset-procedure-not-performed-reason.txt
Embedded valueset-procedure-not-performed-reason.txt
Call valueset-legal-status-of-supply.txt
Embedded valueset-legal-status-of-supply.txt
Call valueset-measure-population.json.txt
Embedded valueset-measure-population.json.txt
Ca

In [3]:
import numpy as np
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
connections.connect("default", host="localhost", port="19530")

fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="filename", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="core", dtype=DataType.BOOL),
    FieldSchema(name="definition", dtype=DataType.BOOL),
    FieldSchema(name="example", dtype=DataType.BOOL),
    FieldSchema(name="valueset", dtype=DataType.BOOL),
    FieldSchema(name="chunk", dtype=DataType.INT16),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=1536)
]
schema = CollectionSchema(fields, "FHIR Spec embedded by openai")
fhir_milvus = Collection("fhir_spec_txt", schema)

pk = 0
with open("./simpler-embedded.json") as embedded_json:
    for l in embedded_json:
        pk += 1
        e = json.loads(l)
        narrative = True if "info" not in e[0] else False
        definition = True if "info" in e[0] else False
        core = True
        example = True if "example" in e[0] else False
        valueset = True if "valueset-" in e[0] else False
        insert_result = fhir_milvus.insert([[pk], [e[0]],[core],[definition],[example],[valueset],  np.array([e[1]]).astype(np.int16), [e[2]]])
# After final entity is inserted, it is best to call flush to have no growing segments left in memory
fhir_milvus.flush()  

# index = {
#     "index_type": "IVF_FLAT",
#     "metric_type": "L2",
#     "params": {"nlist": 128},
# }
# fhir_milvus.create_index(field_name="embeddings", index_params=index)



Status(code=0, message=)

In [4]:
fhir_milvus.release()
fhir_milvus.drop_index()

index = {
    "index_type": "FLAT",
    "metric_type": "L2",
    "params": {},
}
fhir_milvus.create_index(field_name="embeddings", index_params=index)
# fhir_milvus.load()


Status(code=0, message=)

In [ ]:
# 

In [ ]:
fhir_milvus.load()
search_params = {"metric_type": "L2", "params": {}, "offset": 0}

query_embedding = openai.Embedding.create(input = ["fhir patient identifier"], model="text-embedding-ada-002")['data'][0]['embedding']

results = fhir_milvus.search(
	data=[query_embedding], 
	anns_field="embeddings", 
	param=search_params,
	output_fields=['filename', 'chunk'],
	limit=5, 
	expr="core==true",
)[0]

for hit in results:
	print(hit.entity)
	chunk = filename_to_chunks[hit.entity.get('filename')][hit.entity.get('chunk')]
	print(chunk)

id: 14421, distance: 0.32674795389175415, entity: {'filename': 'fhirpath.txt', 'chunk': 12}
fhirpath.txt
generated on Sat, May 28, 2022 12:58+1000.
Links: Search [] Version History Table of Contents QA Page |
Compare to R4 [] [CC0] Propose a change []

id: 3767, distance: 0.3538467586040497, entity: {'filename': 'qa.txt', 'chunk': 47}
qa.txt
 http://hl7.org/fhir/ig/vhdir/Organization/independencerehab in
 orgrole-example-services (Possible Ids: hl7, 1,
 1832473e-2fe0-452d-abe9-3cdb9879522f, 2, 2.16.840.1.113883.19.5,
 f001, f002, f201, f203, f003, mmanu, 3, hl7pay, )
 orgrole-example-services) Unable to resolve invalid example
 reference http://hl7.org/fhir/ig/vhdir/Network/patriotppo in
 orgrole-example-services

Resource Patient

 INFORMATION:Patient.generalPractitioner (example
 patient-example-infant-mom) Unable to resolve example reference to
 Practitioner/21B in patient-example-infant-mom (Possible Ids:
 example, xcda-author, 3ad0687e-f477-468c-afd5-fcc2bf897809, f001,
 f002, f00

In [ ]:
import json
import pandas as pd
import re
import os
from os import walk
from openai.embeddings_utils import get_embedding, get_embeddings
import openai
openai.api_key = os.environ['OPENAI_API_KEY']


fhir_txt_filenames = []
for (dirpath, dirnames, filenames) in walk("./txt-simple/"):
    fhir_txt_filenames.extend([f for f in filenames])
    break

CHUNK_MAX = 4000
def chunk_file(filename, contents):
    contents_clamped = []
    for c in contents:
        for i in range(0, len(c), CHUNK_MAX):
            contents_clamped.append(c[i*CHUNK_MAX:(i+1)*CHUNK_MAX])
    contents = contents_clamped
    def new_chunk(c):
        chunks.append(f'{filename}\n{c}')
    chunks = []
    new_chunk("")
    start_line = 0
    for i, l in enumerate(contents):
        if re.match("^(\d+\.\d+)|Raw JSON", l):
            start_line = i
            break

    for c in contents[start_line:]:
        if len(chunks[-1]) + len(c) > CHUNK_MAX or re.match("^(\d+\.\d+)|Raw JSON", c):
            new_chunk(c)
        else:
            chunks[-1] += c
    return chunks[1:]

filename_to_chunks = {}
for fname in fhir_txt_filenames:
    with open(f'./txt-simple/{fname}') as fhandle:
        contents = fhandle.readlines()
        chunks = chunk_file(fname, contents)
        filename_to_chunks[fname] = chunks

with open("./chunks.json", "w") as chunks_file:
    json.dump(filename_to_chunks, chunks_file, indent=2)

with open("./simpler-embedded.json", "w") as embedded_json:
    for filename in fhir_txt_filenames:
        embeddings = []
        chunks = filename_to_chunks[filename]
        if len(chunks) == 0: continue
        # print(chunks)
        print("Call", filename)
        embed_data = openai.Embedding.create(input = chunks, model="text-embedding-ada-002")['data']
        chunks_embedded = [e['embedding'] for e in embed_data]
        for v in [[filename, i, embedding] for i, embedding in enumerate(chunks_embedded)]:
            json.dump(v, embedded_json)
            embedded_json.write("\n")
        print("Embedded", filename)


Call valueset-all-distance-units.txt
Embedded valueset-all-distance-units.txt
Call sc-valueset-medicationknowledge-status.json.txt
Embedded sc-valueset-medicationknowledge-status.json.txt
Call valueset-encounter-type.json.txt
Embedded valueset-encounter-type.json.txt
Call valueset-condition-stage.txt
Embedded valueset-condition-stage.txt
Call valueset-verificationresult-status.json.txt
Embedded valueset-verificationresult-status.json.txt
Call catalogentry-definitions.txt
Embedded catalogentry-definitions.txt
Call healthcareservice-mappings.txt
Embedded healthcareservice-mappings.txt
Call hlaresult.txt
Embedded hlaresult.txt
Call medicationadministration-mappings.txt
Embedded medicationadministration-mappings.txt
Call valueset-procedure-not-performed-reason.txt
Embedded valueset-procedure-not-performed-reason.txt
Call valueset-legal-status-of-supply.txt
Embedded valueset-legal-status-of-supply.txt
Call valueset-measure-population.json.txt
Embedded valueset-measure-population.json.txt
Ca

In [ ]:
import numpy as np
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
connections.connect("default", host="localhost", port="19530")

fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="filename", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="core", dtype=DataType.BOOL),
    FieldSchema(name="definition", dtype=DataType.BOOL),
    FieldSchema(name="example", dtype=DataType.BOOL),
    FieldSchema(name="valueset", dtype=DataType.BOOL),
    FieldSchema(name="chunk", dtype=DataType.INT16),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=1536)
]
schema = CollectionSchema(fields, "FHIR Spec embedded by openai")
fhir_milvus = Collection("fhir_spec_txt", schema)

pk = 0
with open("./simpler-embedded.json") as embedded_json:
    for l in embedded_json:
        pk += 1
        e = json.loads(l)
        core = True if "-" not in e[0] else False
        definition = True if "definition" in e[0] else False
        example = True if "example" in e[0] else False
        valueset = True if "valueset-" in e[0] else False
        insert_result = fhir_milvus.insert([[pk], [e[0]],[core],[definition],[example],[valueset],  np.array([e[1]]).astype(np.int16), [e[2]]])
# After final entity is inserted, it is best to call flush to have no growing segments left in memory
fhir_milvus.flush()  

index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
fhir_milvus.create_index(field_name="embeddings", index_params=index)



Status(code=0, message=)

In [ ]:
fhir_milvus.release()
fhir_milvus.drop_index()

index = {
    "index_type": "FLAT",
    "metric_type": "L2",
    "params": {},
}
fhir_milvus.create_index(field_name="embeddings", index_params=index)
# fhir_milvus.load()


Status(code=0, message=)

In [ ]:
# 

In [ ]:
fhir_milvus.load()
search_params = {"metric_type": "L2", "params": {}, "offset": 0}

query_embedding = openai.Embedding.create(input = ["fhir patient identifier"], model="text-embedding-ada-002")['data'][0]['embedding']

results = fhir_milvus.search(
	data=[query_embedding], 
	anns_field="embeddings", 
	param=search_params,
	output_fields=['filename', 'chunk'],
	limit=5, 
	expr="core==true",
)[0]

for hit in results:
	print(hit.entity)
	chunk = filename_to_chunks[hit.entity.get('filename')][hit.entity.get('chunk')]
	print(chunk)

id: 14421, distance: 0.32674795389175415, entity: {'filename': 'fhirpath.txt', 'chunk': 12}
fhirpath.txt
generated on Sat, May 28, 2022 12:58+1000.
Links: Search [] Version History Table of Contents QA Page |
Compare to R4 [] [CC0] Propose a change []

id: 3767, distance: 0.3538467586040497, entity: {'filename': 'qa.txt', 'chunk': 47}
qa.txt
 http://hl7.org/fhir/ig/vhdir/Organization/independencerehab in
 orgrole-example-services (Possible Ids: hl7, 1,
 1832473e-2fe0-452d-abe9-3cdb9879522f, 2, 2.16.840.1.113883.19.5,
 f001, f002, f201, f203, f003, mmanu, 3, hl7pay, )
 orgrole-example-services) Unable to resolve invalid example
 reference http://hl7.org/fhir/ig/vhdir/Network/patriotppo in
 orgrole-example-services

Resource Patient

 INFORMATION:Patient.generalPractitioner (example
 patient-example-infant-mom) Unable to resolve example reference to
 Practitioner/21B in patient-example-infant-mom (Possible Ids:
 example, xcda-author, 3ad0687e-f477-468c-afd5-fcc2bf897809, f001,
 f002, f00